In [9]:
import pandas as pd
data = {
    "transaction_id": ["T1001", "T1002", "T1003", "T1004", "T1005"],
    "customer_id": ["C001", "C002", "C003", "C002", "C001"],
    "payment_amount": [125.50, 89.99, 120.00, 54.30, 210.20],
    "payment_date": [
        "2021-10-05",
        "2021-10-06",
        "2021-10-07",
        "2021-10-05",
        "2021-10-08",
    ],
    "payment_status": ["Paid", "Unpaid", "Paid", "Paid", "Pending"],
}

df = pd.DataFrame(data)
df

,transaction_id,customer_id,payment_amount,payment_date,payment_status
0,T1001,C001,125.50,2021-10-05,Paid
1,T1002,C002,89.99,2021-10-06,Unpaid
2,T1003,C003,120.00,2021-10-07,Paid
3,T1004,C002,54.30,2021-10-05,Paid
4,T1005,C001,210.20,2021-10-08,Pending


In [10]:
data = """
    "transaction_id": ["T1001", "T1002", "T1003", "T1004", "T1005"],
    "customer_id": ["C001", "C002", "C003", "C002", "C001"],
    "payment_amount": [125.50, 89.99, 120.00, 54.30, 210.20],
    "payment_date": [
        "2021-10-05",
        "2021-10-06",
        "2021-10-07",
        "2021-10-05",
        "2021-10-08",
    ],
    "payment_status": ["Paid", "Unpaid", "Paid", "Paid", "Pending"],
}
"""
transaction_id = "T1001"

prompt = f"""
Given the following data, what is the payment status for \
transaction_id={transaction_id}? just give accurate answer in object format \
not provide any extra info & details.

data:
{data}

"""

In [11]:
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(model_name="mistral-small-latest")

response = llm.invoke(prompt)

response.content

'{\n  "transaction_id": "T1001",\n  "payment_status": "Paid"\n}'

### Specific Functions

In [19]:
from langchain_core.messages import HumanMessage, ToolMessage
from langchain_core.tools import tool

@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

In [21]:
tools = [add, multiply]
llm_with_tools = llm.bind_tools(tools)

query = "What is 3 * 12? Also, what is 11 + 49?"
messages = [HumanMessage(query)]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '6nV0KqBnu', 'type': 'function', 'function': {'name': 'multiply', 'arguments': '{"a": 3, "b": 12}'}}, {'id': 'u3riigX15', 'type': 'function', 'function': {'name': 'add', 'arguments': '{"a": 11, "b": 49}'}}]}, response_metadata={'token_usage': {'prompt_tokens': 202, 'total_tokens': 251, 'completion_tokens': 49}, 'model': 'mistral-small-latest', 'finish_reason': 'tool_calls'}, id='run-1238a8ab-11c5-468b-9673-e9d1e93e4b46-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': '6nV0KqBnu', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'u3riigX15', 'type': 'tool_call'}], usage_metadata={'input_tokens': 202, 'output_tokens': 49, 'total_tokens': 251}),
 ToolMessage(content='36', tool_call_id='6nV0KqBnu'),
 ToolMessage(content='60', tool_call_id='u3riigX15')]

In [22]:
llm_with_tools.invoke(messages)

AIMessage(content='3 * 12 equals 36.\n\nAnd 11 + 49 equals 60.', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 324, 'total_tokens': 350, 'completion_tokens': 26}, 'model': 'mistral-small-latest', 'finish_reason': 'stop'}, id='run-46a13bf6-37c8-45a1-84bd-4ae4dc0ac6ff-0', usage_metadata={'input_tokens': 324, 'output_tokens': 26, 'total_tokens': 350})